## Accessing Copernicus Data Space Ecosystem (CDSE) EODATA via S3

First of all, you need to configure **AWS CLI** and **mount the EODATA bucket** from the Copernicus Data Space Ecosystem (CDSE).


#### 1. Generate Access Keys
Visit the [CDSE S3 Keys Manager](https://eodata-s3keysmanager.dataspace.copernicus.eu/) and generate your **Access Key** and **Secret Key**.


#### 2. Configure AWS CLI
Follow the [official instructions](https://documentation.dataspace.copernicus.eu/APIs/S3.html#accessing-eodata-via-aws-cli).


#### 3. Mount EODATA with s3fs

1.  Create a file called `~/.passwd_dataspace_copernicus` that contains your keys in the following format:

       your_access_key:your_secret_key

3.  Create a local mount point:

    ``` bash
    sudo mkdir -p /eodata && sudo chmod +777 /eodata && chmod 600 /.../.passwd_dataspace_copernicus
    ```


4.  Mount the EODATA bucket:

    ``` bash
    (fusermount -u /eodata || echo ok) && s3fs EODATA /eodata -o passwd_file=/.../.passwd_dataspace_copernicus -o use_cache=/tmp -o host=https://eodata.dataspace.copernicus.eu -o endpoint=RegionOne -o use_path_request_style -f
    ```

5.  Verify the mount:

    ``` bash
    ls /eodata
    ```

----

Now we can proceed by importing all necessary libraries.

In [2]:
import pystac
from pystac_client import Client
import odc.stac

Set up the STAC client and connect to the Copernicus STAC API.

In [3]:
STAC_URL = "https://stac.dataspace.copernicus.eu/v1/"
client = Client.open(STAC_URL)

Define the bounding box, the date range and query the desired collection (Sentinel-2 L1C in this example).

In [5]:
stac_items = client.search(
    collections=['sentinel-2-l1c'],
    bbox=[-70.3,-28.8,-69.8,-28.2],
    datetime=["2023-01-02", "2023-01-10"]
).item_collection()

Modify asset URLs to point to local /eodata mount. This step updates the "href" of all assets so they point to your mounted EODATA folder instead of remote URLs.

In [6]:
new_stac_items = []
for item in stac_items:
    item_dict = item.to_dict()
    
    for asset in item_dict["assets"]:
        item_dict["assets"][asset]["href"] = item_dict["assets"][asset]["href"][4:]
        
    new_stac_items.append(pystac.Item.from_dict(item_dict))

Load the STAC items into an ODC xarray dataset

In [7]:
ds = odc.stac.load(new_stac_items, chunks={})
ds

<xarray.Dataset> Size: 26GB
Dimensions:      (y: 10980, x: 20976, time: 2)
Coordinates:
  * y            (y) float64 88kB 6.9e+06 6.9e+06 6.9e+06 ... 6.79e+06 6.79e+06
  * x            (x) float64 168kB 3e+05 3e+05 3e+05 ... 5.097e+05 5.098e+05
    spatial_ref  int32 4B 32719
  * time         (time) datetime64[ns] 16B 2023-01-03T14:37:29.024000 2023-01...
Data variables: (12/14)
    B01          (time, y, x) float32 2GB dask.array<chunksize=(1, 10980, 20976), meta=np.ndarray>
    B02          (time, y, x) float32 2GB dask.array<chunksize=(1, 10980, 20976), meta=np.ndarray>
    B03          (time, y, x) float32 2GB dask.array<chunksize=(1, 10980, 20976), meta=np.ndarray>
    B04          (time, y, x) float32 2GB dask.array<chunksize=(1, 10980, 20976), meta=np.ndarray>
    B05          (time, y, x) float32 2GB dask.array<chunksize=(1, 10980, 20976), meta=np.ndarray>
    B06          (time, y, x) float32 2GB dask.array<chunksize=(1, 10980, 20976), meta=np.ndarray>
    ...           ...
    B09          (time, y, x) float32 2GB dask.array<chunksize=(1, 10980, 20976), meta=np.ndarray>
    B11          (time, y, x) float32 2GB dask.array<chunksize=(1, 10980, 20976), meta=np.ndarray>
    B12          (time, y, x) float32 2GB dask.array<chunksize=(1, 10980, 20976), meta=np.ndarray>
    B8A          (time, y, x) float32 2GB dask.array<chunksize=(1, 10980, 20976), meta=np.ndarray>
    TCI          (time, y, x) float32 2GB dask.array<chunksize=(1, 10980, 20976), meta=np.ndarray>
    B10          (time, y, x) float32 2GB dask.array<chunksize=(1, 10980, 20976), meta=np.ndarray>

Access a small slice.

In [8]:
band_slice =ds.B01[:10,:10,1].values
band_slice

array([[2554., 2554., 2554., 2554., 2554., 2554., 2537., 2537., 2537.,
        2537.],
       [2401., 2401., 2401., 2401., 2401., 2401., 2380., 2380., 2380.,
        2380.]], dtype=float32)